In [1]:
import tensorflow._api.v2.compat.v1 as tf
import tensorflow_hub as hub
import numpy as np
import pickle 

tf.disable_v2_behavior()
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.ERROR)

WILDCARD_CHAR = '\t'
MAX_LEN = 16
CHARMAP_PATH = 'DATA/AE_char_map.pickle'
MODEL_PATH = 'DATA/AE_based_Noise_a4_ls128_hn4_al8/'

2023-11-04 23:29:17.490119: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
template = 'p***r'
template = template.replace('*', WILDCARD_CHAR)
template

'p\t\t\tr'

In [3]:
with open(CHARMAP_PATH, 'rb') as f:
    char2id: dict[str, int] = pickle.load(f)
char2id

{'\n': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'p': 10,
 'a': 11,
 's': 12,
 'w': 13,
 'o': 14,
 'r': 15,
 'd': 16,
 'i': 17,
 'l': 18,
 'v': 19,
 'e': 20,
 'y': 21,
 'u': 22,
 'n': 23,
 'c': 24,
 'k': 25,
 'b': 26,
 'g': 27,
 'm': 28,
 'j': 29,
 'h': 30,
 'q': 31,
 't': 32,
 '0': 33,
 'f': 34,
 'z': 35,
 'x': 36,
 'P': 37,
 'A': 38,
 'S': 39,
 'W': 40,
 'O': 41,
 'R': 42,
 'D': 43,
 '!': 44,
 'I': 45,
 'L': 46,
 'V': 47,
 'E': 48,
 'Y': 49,
 'U': 50,
 'N': 51,
 'C': 52,
 ' ': 53,
 ';': 54,
 'B': 55,
 'K': 56,
 '-': 57,
 'G': 58,
 'J': 59,
 'H': 60,
 'T': 61,
 'M': 62,
 '*': 63,
 '.': 64,
 '?': 65,
 ',': 66,
 '/': 67,
 'Ã': 68,
 '±': 69,
 'Q': 70,
 '#': 71,
 'F': 72,
 'X': 73,
 '@': 74,
 '$': 75,
 '%': 76,
 '^': 77,
 'Z': 78,
 '&': 79,
 '+': 80,
 "'": 81,
 '[': 82,
 ']': 83,
 '<': 84,
 '_': 85,
 '\\': 86,
 '"': 87,
 'Â': 88,
 '£': 89,
 ':': 90,
 '(': 91,
 ')': 92,
 '=': 93,
 'à': 94,
 '¹': 95,
 '\x85': 96,
 '¸': 97,
 '\xa0': 98,
 '\x96': 99,

In [4]:
id2char = [''] * len(char2id)
for char, id in char2id.items():
    id2char[id] = char
id2char

['\n',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'p',
 'a',
 's',
 'w',
 'o',
 'r',
 'd',
 'i',
 'l',
 'v',
 'e',
 'y',
 'u',
 'n',
 'c',
 'k',
 'b',
 'g',
 'm',
 'j',
 'h',
 'q',
 't',
 '0',
 'f',
 'z',
 'x',
 'P',
 'A',
 'S',
 'W',
 'O',
 'R',
 'D',
 '!',
 'I',
 'L',
 'V',
 'E',
 'Y',
 'U',
 'N',
 'C',
 ' ',
 ';',
 'B',
 'K',
 '-',
 'G',
 'J',
 'H',
 'T',
 'M',
 '*',
 '.',
 '?',
 ',',
 '/',
 'Ã',
 '±',
 'Q',
 '#',
 'F',
 'X',
 '@',
 '$',
 '%',
 '^',
 'Z',
 '&',
 '+',
 "'",
 '[',
 ']',
 '<',
 '_',
 '\\',
 '"',
 'Â',
 '£',
 ':',
 '(',
 ')',
 '=',
 'à',
 '¹',
 '\x85',
 '¸',
 '\xa0',
 '\x96',
 '§',
 '`',
 '\x91',
 '~',
 '>',
 '³',
 '©',
 '¼',
 '{',
 '}',
 '\x9f',
 'Å',
 'Ä',
 '|',
 '¶',
 '´',
 'ñ',
 '×',
 '\x97',
 '\x95',
 '\x94',
 '¡',
 '¤',
 '¥',
 'å',
 'æ',
 'ø',
 'â',
 '\x99',
 '\x88',
 '\x81',
 '¢',
 '\x86',
 '\x84',
 '\x9e',
 '°',
 '²',
 '·',
 'ª',
 '\xad',
 '«',
 'Ù',
 '\x8a',
 'Ø',
 'Î',
 '»',
 '¿',
 'Ï',
 '\x82',
 '\x80',
 '¬',
 'µ',
 '¦',
 'º',
 '¨',
 'é',
 '\x93',

In [5]:
template_as_vector = np.zeros(MAX_LEN, dtype=np.int32)
for i, c in enumerate(template): 
    template_as_vector[i] = char2id.get(c, -1)
template_as_vector = template_as_vector[None, :]
template_as_vector

array([[10, -1, -1, -1, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int32)

In [6]:
module = hub.Module(MODEL_PATH)
x_placeholder = tf.placeholder(tf.int32, shape=(None, MAX_LEN))

# number of samples
n = 10000
# standard devation
stddev = 0.6

module_inputs = {'x': x_placeholder, 'stddev': (stddev,), 'n': (n,)}
module_dict = module(module_inputs, as_dict=True, signature='sample_from_latent')
prediction_tensor = module_dict['prediction']

In [7]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    samples = sess.run(prediction_tensor, {x_placeholder: template_as_vector})
samples.shape

2023-11-04 23:29:22.974270: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled


(10000, 16)

In [8]:
samples_as_strings = list(map(
    lambda vec: ''.join(id2char[i] for i in vec if i > 0),
    samples
))
# samples_as_strings

In [9]:
samples_filtered = list(set(filter(
    lambda sample: len(sample) == len(template)
                   and all(sc == tc or tc == WILDCARD_CHAR
                           for sc, tc in zip(sample, template)),
    samples_as_strings
)))
# samples_filtered

In [10]:
len(samples_as_strings), len(samples_filtered)

(10000, 204)

In [13]:
'peter' in samples_filtered

False

In [12]:
samples_filtered

['piorr',
 'paler',
 'pecar',
 'papar',
 'peser',
 'pekor',
 'pever',
 'pucer',
 'pster',
 'peier',
 'piser',
 'pieer',
 'pogar',
 'puper',
 'peyer',
 'piwer',
 'puier',
 'piplr',
 'putrr',
 'peper',
 'piger',
 'psker',
 'piker',
 'puwer',
 'pellr',
 'pesrr',
 'pewar',
 'pit0r',
 'piber',
 'pebar',
 'pilar',
 'peeer',
 'paaer',
 'p3t1r',
 'picer',
 'poaer',
 'pigor',
 'pikrr',
 'pusar',
 'pucar',
 'poser',
 'pecer',
 'pator',
 'puber',
 'pitar',
 'pecur',
 'pipar',
 'pivor',
 'p3sor',
 'puerr',
 'puarr',
 'pibar',
 'peear',
 'pusor',
 'pewer',
 'pioer',
 'petar',
 'peorr',
 'pitrr',
 'popar',
 'piqer',
 'pacor',
 'piper',
 'piter',
 'paoer',
 'patar',
 'pearr',
 'popor',
 'peder',
 'pegar',
 'paarr',
 'pswar',
 'papor',
 'pieor',
 'peger',
 'pupor',
 'pugar',
 'pesar',
 'puger',
 'p2g3r',
 'piaer',
 'pealr',
 'puser',
 'petur',
 'pawer',
 'psper',
 'pevor',
 'pherr',
 'poper',
 'pstrr',
 'piner',
 'peker',
 'puker',
 'putur',
 'polar',
 'pooor',
 'pacer',
 'powar',
 'poter',
 'poler',
